In [1]:
## Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")

import os.path
import pandas
import math
import time

from metrics.Correlation import Correlation
from IO.Write import Write
from IO.Read import Read
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)
import DataSet.SnpsSelection as s

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import DenseVector

In [2]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "20g")
       .set('spark_executor_cores',"3"))

#spark = SparkSession.builder.appName('melanoma').getOrCreate()
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [3]:
#spark.stop()

# Tests

In [4]:
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.01/snp2.txt'
pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'

#pathSnp = '/media/antonis/Antonis_Moulopoulos/newSet/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/Antonis_Moulopoulos/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'

data = spark.read.option("maxColumns", 80000).csv(pathSnp,inferSchema=True,header=True)
data=data.withColumnRenamed('TARGET','label')

In [5]:
features = []
for i in data.columns:
    if 'rs' not in i and i !='label':
        features.append(i)
print(len(features))
#print((features))

141


In [6]:
#data.select('patients','label').show(10)

In [7]:
from pyspark.ml.linalg import DenseVector

d = data.drop('patients')
for i in features:
    d = d.drop(i)
print("columns = ",len(d.columns))
li = len(d.columns)
input_data = d.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))

df = spark.createDataFrame(input_data, ["label", "features"])

columns =  12003


In [8]:
df.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[0.0,0.0,0.0,0.0,...|
|    0|[0.0,0.0,0.0,0.0,...|
|    0|[0.0,0.0,2.0,1.0,...|
|    0|[0.0,0.0,1.0,1.0,...|
|    1|[0.0,1.0,1.0,0.0,...|
|    0|[1.0,0.0,1.0,0.0,...|
|    0|[0.0,1.0,1.0,0.0,...|
|    0|[1.0,0.0,1.0,0.0,...|
|    0|[1.0,1.0,1.0,0.0,...|
|    0|[1.0,1.0,1.0,1.0,...|
+-----+--------------------+
only showing top 10 rows



# Correlation

In [9]:
from pyspark.ml.stat import Correlation
final_data1 = df.select('features')
corr = Correlation.corr(final_data1, "features")
corr = corr.head()[0].toArray()

In [10]:
snpsRed = []
snpsRed = s.lowCorrelation(corr, threshold=0.7, up=100, down=99)

count =  9470
len snpsRed =  9470


In [11]:

d = data.drop('patients')
features = []

for i in data.columns:
    if 'rs' in i and i !='label':
        features.append(i)
print(len(features))

#features = d.columns

print("len features1 = ",len(features))

snpsRed1 = []
for i in snpsRed:
    snpsRed1.append(features[i])

snpsRed1.append('label')
    
features = []
for i in data.columns:
    if i not in snpsRed1:
        features.append(i)
print("len features2 = ",len(features))

for i in features:
    d = d.drop(i)
print("columns = ",len(d.columns))
li = len(d.columns)
input_data = d.rdd.map(lambda x: (x[li-1], DenseVector(x[:li-1])))

df = spark.createDataFrame(input_data, ["label", "features"])

#assempler = VectorAssembler(inputCols=snpsRed1,outputCol='snpsRed')
#output = assempler.transform(data)
#final_data = output.select('snpsRed','label')
#final_data.show(10)

12002
len features1 =  12002
len features2 =  2673
columns =  9471


In [12]:
#train_data,test_data = final_data.randomSplit([0.9,0.1],seed=11)

train_data,test_data = df.randomSplit([0.9,0.1],seed=11)

train_data=train_data.withColumnRenamed('TARGET','label')
test_data=test_data.withColumnRenamed('TARGET','label')

In [13]:
train0,t2 = train_data.filter('label == 0').randomSplit([0.001,0.1],seed = 11)
train1 = train_data.filter("label == 1")
#t0 = train0.count()
#t1 = train1.count()

In [14]:
train0 = train0.union(train1)

In [15]:
log_reg = LogisticRegression(featuresCol='features',labelCol='label',maxIter=10)
lr_model = log_reg.fit(train0)

In [16]:
#print("t0 = ",t0)
#print("t1 = ",t1)
#print("train0", train0.count())
#print("all =", t0+t1)

#print("final = ",df.count())
#print("test_data = ", test_data.count())
#print("train_data = ", train_data.count())
#print("all = ", test_data.count() + train_data.count())

In [17]:
results = lr_model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

0.8249659619542328

In [18]:
results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[0.0,0.0,0.0,0.0,...|[0.44220094195598...|[0.60878334601552...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[5.05137818050948...|[0.99364019963049...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[2.56100442277661...|[0.92830933195703...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[5.86288126257737...|[0.99716501963883...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[-0.7378737329609...|[0.32346927469249...|       1.0|
|    0|[0.0,0.0,0.0,0.0,...|[3.60328917335768...|[0.97348802931815...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[-0.8540668432135...|[0.29858044082104...|       1.0|
|    0|[0.0,0.0,0.0,0.0,...|[10.6547725584097...|[0.99997641255894...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[4.25464560894296...|[0.98600064303036...|       0.0|
|    0|[0.0,0.0,

In [19]:
'''Param for metric name in evaluation. Supports: - "rmse" (default): root mean squared error - 
"mse": mean squared error - "r2": R^2^ metric - "mae": mean absolute error '''

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

#lr = LinearRegression(maxIter=maxIteration)
#pipeline = Pipeline(stages=[output, log_reg])
pipeline = Pipeline(stages=[log_reg])
#fit_model = pipeline.fit(train_data)
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label')
paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=log_reg,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=4)

cvModel = crossval.fit(train0)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
#results.show(10)

In [20]:
cross_results

DataFrame[label: bigint, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [55]:
from pyspark.mllib.evaluation import MulticlassMetrics


c = cross_results.rdd
predictionAndLabels = c.map(lambda lp: (lp.prediction, float(lp.label)))
metrics = MulticlassMetrics(predictionAndLabels)

In [56]:
metrics.confusionMatrix().toArray()

array([[20089.,  3892.],
       [   43.,   186.]])

In [51]:
print("accuracy = ", metrics.accuracy)
print("recal = ", metrics.recall(0))
print("precision = ", metrics.precision(0))
print("f1 = ", metrics.fMeasure(1.0))

accuracy =  0.8374638579099546
recal =  0.9978640969600636
precision =  0.8377048496726576
f1 =  0.08637102391455768


In [39]:
r = results.rdd
r3 = r.collect()

In [48]:
same0 = 0
same1 = 0
sum0 = 0
sum1 = 0
sumall = 0
for i in r3:
    
    if i[1] == 0:
        sum0 += 1
        if i[0] == 0:
            same0 += 1
    elif i[1] == 1:
        sum1 += 1
        if i[0] == 1:
            same1 += 1
    sumall += 1

In [49]:
print('sum0 = ', sum0)
print('sum1 = ', sum1)
print('same0 = ', same0)
print('same1 = ', same1)
print('all = ', sumall)
print('all2 = ', sum0+sum1)

sum0 =  23981
sum1 =  229
same0 =  20089
same1 =  186
all =  24210
all2 =  24210


In [45]:
test_data.filter("label==1").count()

229

In [28]:
cross_results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[0.0,0.0,0.0,0.0,...|[0.44220094195599...|[0.60878334601552...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[5.05137818050953...|[0.99364019963049...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[2.56100442277666...|[0.92830933195704...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[5.86288126257740...|[0.99716501963883...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[-0.7378737329609...|[0.32346927469249...|       1.0|
|    0|[0.0,0.0,0.0,0.0,...|[3.60328917335766...|[0.97348802931815...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[-0.8540668432135...|[0.29858044082104...|       1.0|
|    0|[0.0,0.0,0.0,0.0,...|[10.6547725584098...|[0.99997641255894...|       0.0|
|    0|[0.0,0.0,0.0,0.0,...|[4.25464560894294...|[0.98600064303036...|       0.0|
|    0|[0.0,0.0,

In [57]:
#print("Coefficients: \n" + str(lr_model.coefficientMatrix.shape))

In [34]:
len(lr_model.coefficients)

9470

In [65]:
lr_model.coefficientMatrix.toArray()

array([[-0.03604068, -0.0529003 , -0.01648724, ..., -0.00118198,
         0.04564904, -0.0080924 ]])

In [60]:
print(len(d.columns))
print(d.columns[len(d.columns)-1])

9471
label


In [64]:
for i in lr_model.coefficientMatrix.toArray()[0]:
    print(i)
    break

-0.03604068230256025
